In [127]:
# Lambda-Werte für die Depots
lambdas = [2, 3, 5]  # lambda1, lambda2, lambda3
lambda_total = sum(lambdas)
probs = [l / lambda_total for l in lambdas]  # p1, p2, p3

# Leadtimes
Lj = 3 / 30  
L0 = 15 / 30  

# Lagerbestände
S0 = 4  
S = [1, 1, 2]  # S1, S2, S3

# Depot auswählen: 0 = Depot 1, 1 = Depot 2, 2 = Depot 3
depot_index = 2  

# Aktuelles Depot
S_j = S[depot_index]
lambda_j = lambdas[depot_index]
p_j = probs[depot_index]


In [128]:
import numpy as np
from scipy.stats import poisson

k_values = np.arange(0, 16)
P_O0 = poisson.pmf(k_values, mu=lambda_total * L0)
O0_probabilities = P_O0  # NumPy Array

for k, p in enumerate(O0_probabilities):
    print(f"P(O0 = {k}) = {p:.4f}")


P(O0 = 0) = 0.0067
P(O0 = 1) = 0.0337
P(O0 = 2) = 0.0842
P(O0 = 3) = 0.1404
P(O0 = 4) = 0.1755
P(O0 = 5) = 0.1755
P(O0 = 6) = 0.1462
P(O0 = 7) = 0.1044
P(O0 = 8) = 0.0653
P(O0 = 9) = 0.0363
P(O0 = 10) = 0.0181
P(O0 = 11) = 0.0082
P(O0 = 12) = 0.0034
P(O0 = 13) = 0.0013
P(O0 = 14) = 0.0005
P(O0 = 15) = 0.0002


In [129]:
# Array für Backorder-Wahrscheinlichkeiten
B0_probabilities = np.zeros(len(O0_probabilities))

# B0 = 0 (keine Backorders)
B0_probabilities[0] = np.sum(O0_probabilities[:S0 + 1])

# B0 = 1, 2, ..., max_backorder-1
for b in range(1, len(O0_probabilities)-1):
    if S0 + b < len(O0_probabilities):
        B0_probabilities[b] = O0_probabilities[S0 + b]
    else:
        B0_probabilities[b] = 0

# Ausgabe
for b, p in enumerate(B0_probabilities):
    print(f"P(B0 = {b}) = {p:.4f}")


P(B0 = 0) = 0.4405
P(B0 = 1) = 0.1755
P(B0 = 2) = 0.1462
P(B0 = 3) = 0.1044
P(B0 = 4) = 0.0653
P(B0 = 5) = 0.0363
P(B0 = 6) = 0.0181
P(B0 = 7) = 0.0082
P(B0 = 8) = 0.0034
P(B0 = 9) = 0.0013
P(B0 = 10) = 0.0005
P(B0 = 11) = 0.0002
P(B0 = 12) = 0.0000
P(B0 = 13) = 0.0000
P(B0 = 14) = 0.0000
P(B0 = 15) = 0.0000


In [130]:
from scipy.stats import binom

b_max = len(B0_probabilities) - 1

# Backorders für das ausgewählte Depot
B0_j_binom_probabilities = np.zeros(len(B0_probabilities))

for m in range(b_max + 1):
    s = 0
    for y in range(m, b_max + 1):
        s += binom.pmf(m, y, p_j) * B0_probabilities[y]
    B0_j_binom_probabilities[m] = s

# Ausgabe
for m, p in enumerate(B0_j_binom_probabilities):
    print(f"P(B0,{depot_index + 1} = {m}) = {p:.4f}")


P(B0,3 = 0) = 0.5834
P(B0,3 = 1) = 0.2243
P(B0,3 = 2) = 0.1176
P(B0,3 = 3) = 0.0497
P(B0,3 = 4) = 0.0176
P(B0,3 = 5) = 0.0054
P(B0,3 = 6) = 0.0015
P(B0,3 = 7) = 0.0003
P(B0,3 = 8) = 0.0001
P(B0,3 = 9) = 0.0000
P(B0,3 = 10) = 0.0000
P(B0,3 = 11) = 0.0000
P(B0,3 = 12) = 0.0000
P(B0,3 = 13) = 0.0000
P(B0,3 = 14) = 0.0000
P(B0,3 = 15) = 0.0000


In [131]:
expected_backorders = sum(m * p for m, p in enumerate(B0_j_binom_probabilities))
print(f"Erwartete Backorders Depot {depot_index + 1}: {expected_backorders:.4f}")


Erwartete Backorders Depot 3: 0.7180


In [132]:
# Erwartete Nachfrage während Leadtime für das Depot
mean_demand_j = lambda_j * Lj

# Maximaler k-Wert für die Verteilung
max_k = int(mean_demand_j + 4 * np.sqrt(mean_demand_j)) + b_max + 1
max_k = max(max_k, 10)

# Nachfrageverteilung D_j(L_j) ~ Poisson
demand_probs = poisson.pmf(np.arange(max_k + 1), mean_demand_j)

# Outstanding Orders Verteilung O_j = D_j(L_j) + B0_j
outstanding_probs = np.zeros(max_k + 1)
for k in range(max_k + 1):
    for y in range(min(k, len(B0_j_binom_probabilities) - 1) + 1):
        d = k - y
        if d < len(demand_probs):
            outstanding_probs[k] += demand_probs[d] * B0_j_binom_probabilities[y]

# Normalisierung
outstanding_probs /= outstanding_probs.sum()

# Ausgabe
for k, p in enumerate(outstanding_probs):
    if p > 0.0001:
        print(f"P(O{depot_index + 1} = {k}) = {p:.4f}")

# Array für spätere Verwendung
O_j_probs = outstanding_probs


P(O3 = 0) = 0.3539
P(O3 = 1) = 0.3130
P(O3 = 2) = 0.1836
P(O3 = 3) = 0.0902
P(O3 = 4) = 0.0384
P(O3 = 5) = 0.0143
P(O3 = 6) = 0.0047
P(O3 = 7) = 0.0014
P(O3 = 8) = 0.0004


In [133]:
# Erwartete Backorders für das Depot
E_B0_j = sum(k * p for k, p in enumerate(B0_j_binom_probabilities))

k_vals = np.arange(S_j + 1)
backorder_contrib = (S_j - k_vals) * O_j_probs[k_vals]
EBO_j = lambda_j * Lj + E_B0_j - S_j + backorder_contrib.sum()

print(f"Erwartete Backorders EBO{depot_index + 1}(S0, S{depot_index + 1}) = {EBO_j:.4f}")


Erwartete Backorders EBO3(S0, S3) = 0.2387
